# BLS API

**Prerequisites**

- APIs

**Outcomes**

- Register for an API key to access data from US Bureau of Labor Statistics (BLS)
- Write Python code that integrates with the BLS API
- Construct a dataset of unemployment data as reporeted by the BLS

**Data**

- All data from monthly unemployment reports from US Bureau of Labor Statistics

## The BLS API

The US Bureau of Labor statistics collects data on employment statistics for the US

They provide this data to the public via an API

The data is available without the need for an API key

However, if we supply an API key we can get additional information in the response

## Registering for an API key

Let's open the [landing page](https://www.bls.gov/developers/) for the BLS API

While there we will look for how to request an API key

Once we recieve the API key via email, we will store it somewhere safe 

Treat it like a password -- in the world of APIs that's what it is!

## Building the request

We now need to use the API documentation to learn how to build a request

Recall that a request has the following components:

- Endpoint
- Request type
- Query Parameters
- Payload
- Headers
- Authentication



We'll find these one at a time below

### Endpoint

Let's go to the API documentation

Our goal here is to find the endpoint where we can access the data

We find the docs by clicking the ["BLS API Signatures"](https://www.bls.gov/developers/api_signature_v2.htm) link on the [landing page](https://www.bls.gov/developers/)

As shown in the docs, our endpoint is `https://api.bls.gov/publicAPI/v2/timeseries/data`

### Request Type and Query Parameters

Let's find the request type

The docs say 

> "HTTP Type: POST", 

meaning we need to use a `POST` request

We can specify all information in the *body* of the request, so no query parameters are needed

###  Payload

The docs indicate the structure of the payload

They say

```json
{
    "seriesid":["Series1",..., "SeriesN"],
    "startyear":"yearX", 
    "endyear":"yearY",
    "catalog":true|false,
    "calculations":true|false,
    "annualaverage":true|false,
    "aspects":true|false,
    "registrationkey":"995f4e779f204473aa565256e8afe73e"
}
```

The required parameters are `seriesid`, `startyear`, and `endyear`

We will also pass `catalog: true` so we can get extra information about each variable

### Headers

The only header we need in our request is `"Content-Type": "application/json"`

This tells the api provider that the payload or body is JSON

> Note: recall JSON is web-speak for a Python dict

### Authentication

To identify ourselves to the API provider, we use our API key

As noted in the payload section above, we set the `registrationkey` field in our payload to the API Key

### Summary

Let's summarize:

- We will make `POST` reqeusts to `https://api.bls.gov/publicAPI/v2/timeseries/data/
- We don't have any query parameters
- We pass `registrationkey`, `seriesid`, `startyear`, `endyear`, and `catalog` in the payload
- And we set headers to `{"Content-Type": "application/json"}` note our payload is JSON

## Python requests

Let's give it a shot!

We'll use the python `requests` package to make the HTTP requests

In [2]:
import requests
import json
import pandas as pd
from typing import List, Union, Optional

### Making the request

Below we define a Python function that makes the request

Let's study it together

In [4]:
def request_for_series(
        series_ids: List[str], 
        startyear: Union[int,str], 
        endyear: Union[int,str],
        apikey:Optional[str]=None,
    ):
    """
    Request data for all ``series_ids`` between ``startyear`` and ``endyear``
    
    Parameters
    ----------
    series_ids: List[str]
        A list of all BLS series IDs for which to request data
    
    startyear, endyear: Union[int,str]
        Starting and ending years for period of data. All intervals between
        these two years (inclusive) will be reported
    
    apikey: Optional[str]
        A registration or API key to enable more extensive use of the
        api and more detailed results
    
    Returns
    -------
    responses: List[requests.Response]
        A list of `Response` objects from the requests library
    
    Notes
    -----
    The BLS API only allows 25 series to be requested in a single call to the API
    This function allows an arbitrary number of series. The function first checks how 
    many series_ids are reuqested, and then makes two recursive calls to this function:
    (1) the first 25 series IDs are fetched and (2) the rest of the series ids. If the
    second request contains more than 25 series, another split is made and a pair of 
    recursive function calls are issued.
    
    Also note that the response objects from ``requests`` are not processed or validated
    in any way -- this is up to the caller of this routine.
    
    Finally, if an apikey is given, then a catalog of series metadata will be requested
    and returned from this function. This is necessary for getting the metadata DataFrame
    from the functions ``make_dfs_from_series``, ``unpack_response``, and 
    ``unpack_all_responses`` functions below.
    """
    n_series = len(series_ids)
    if n_series > 25:
        parts = []
        # make common keyword arguments so we don't have to type twice below
        kw = dict(endyear=endyear, startyear=startyear, apikey=apikey)
        parts.extend(request_for_series(series_ids[:25], **kw))
        parts.extend(request_for_series(series_ids[25:], **kw))
        return parts
    else:
        headers = {'Content-type': 'application/json'}
        params = {
            "seriesid": series_ids, 
            "startyear":startyear, 
            "endyear": endyear,
        }
        if apikey is not None:
            params["catalog"] = True
            params["registrationkey"] = apikey
        
        # convert params dictionary to json string
        data = json.dumps(params)
        p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
        return [p]

Let's make a request

In [13]:
# These lines are to keep my api key private, they likely won't work for you!
# you should replace right hand side of `apikey = ` with a string containing
# your api key!
import os
apikey = "061b1e15424740ee915239d8b469e157"
# os.environ.get("BLS_KEY", None)

# make the request
responses = request_for_series(["LAUCN040010000000005"], 2010, 2020, apikey)

In [14]:
type(responses[0])

requests.models.Response

### Validate responses

Our `request_for_series` function will make the `POST` requests and return a list of responses

It does not, however, check to see if the requests were successful

We'll now write some code to verify that requests were successful

#### HTTP codes

Part of the HTTP standard is that a response will include an integer that is the response code

Codes are three digits

Successful resposnes are between 200 and 299

The next function checks that the request code is no more than 299

In [15]:
def check_response(res: requests.Response):
    """
    Check a response from the BLS API for success
    
    Parameters
    ----------
    res: requests.Response
        The requests object returned from iteracting with BLS API
        
    Notes
    -----
    Right now we just check for success at the http protocol level
    and don't do any checking specific to the BLS api
    """
    code = res.status_code
    if code > 299:
        raise ValueError(f"Response error with code {code}")

We can apply our function to the response we recieved

In [16]:
check_response(responses[0])

It did not return or do anything, which is great!

The function would have raised an error if there was a problem

### Unpacking the data

Let's take a closer look at the data that was returned to us

The `requests.Response` type has a `.content` field that includes the data returned from the API provider:

In [17]:
responses[0].headers

{'Date': 'Wed, 14 Oct 2020 04:26:20 GMT', 'Set-Cookie': 'JSESSIONID=9CB41D270D19255F11311040E7254180; Path=/publicAPI; HttpOnly;Version=1;Secure', 'Content-Type': 'application/json', 'Vary': 'Origin, Accept-Encoding', 'APC-Route-id': '_02_09v', 'Access-Control-Allow-Headers': 'origin,x-requested-with,access-control-request-headers,content-type,access-control-request-method,accept', 'Access-Control-Allow-Origin': '*', 'Route-Id': '_t8_07v; ', 'Keep-Alive': 'timeout=5, max=90', 'Connection': 'Keep-Alive', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains', 'Pool-Info': 'AP02', 'Content-Encoding': 'gzip', 'Content-Length': '1541'}

In [18]:
responses[0].content 

b'{"status":"REQUEST_SUCCEEDED","responseTime":177,"message":[],"Results":{\n"series":\n[{"seriesID":"LAUCN040010000000005","catalog":{"series_title":"Employment: Apache County, AZ (U)","series_id":"LAUCN040010000000005","seasonality":"Not Seasonally Adjusted","survey_name":"Local Area Unemployment Statistics","survey_abbreviation":"LA","measure_data_type":"employment","area":"Apache County, AZ","area_type":"Counties and equivalents"},"data":[{"year":"2020","period":"M08","periodName":"August","latest":"true","value":"17301","footnotes":[{"code":"P","text":"Preliminary."}]},{"year":"2020","period":"M07","periodName":"July","value":"15976","footnotes":[{}]},{"year":"2020","period":"M06","periodName":"June","value":"16476","footnotes":[{}]},{"year":"2020","period":"M05","periodName":"May","value":"17415","footnotes":[{}]},{"year":"2020","period":"M04","periodName":"April","value":"16923","footnotes":[{}]},{"year":"2020","period":"M03","periodName":"March","value":"18198","footnotes":[{}]

It appears that this is JSON

We can call the `.json()` method to have `requests` read the `content` into a dict

In [19]:
js = responses[0].json()
js

{'status': 'REQUEST_SUCCEEDED',
 'responseTime': 177,
 'message': [],
 'Results': {'series': [{'seriesID': 'LAUCN040010000000005',
    'catalog': {'series_title': 'Employment: Apache County, AZ (U)',
     'series_id': 'LAUCN040010000000005',
     'seasonality': 'Not Seasonally Adjusted',
     'survey_name': 'Local Area Unemployment Statistics',
     'survey_abbreviation': 'LA',
     'measure_data_type': 'employment',
     'area': 'Apache County, AZ',
     'area_type': 'Counties and equivalents'},
    'data': [{'year': '2020',
      'period': 'M08',
      'periodName': 'August',
      'latest': 'true',
      'value': '17301',
      'footnotes': [{'code': 'P', 'text': 'Preliminary.'}]},
     {'year': '2020',
      'period': 'M07',
      'periodName': 'July',
      'value': '15976',
      'footnotes': [{}]},
     {'year': '2020',
      'period': 'M06',
      'periodName': 'June',
      'value': '16476',
      'footnotes': [{}]},
     {'year': '2020',
      'period': 'M05',
      'periodNa

Notice the structure of the response:

```json
{
    'status': 'REQUEST_SUCCEEDED',
    'responseTime': 190,
    'message': [],
    'Results': {
        'series': [
            {
                'seriesID': 'LAUCN040010000000005',
                'catalog': {
                    'series_title': 'Employment: Apache County, AZ (U)',
                    'series_id': 'LAUCN040010000000005',
                    ...
                },
                'data': [
                    ...
                ]
            }
        ]

    }
}
```



A few points:

- The results are in `js["Results"]["series"]`
- This contains a list of dictionaries, each one having
    - `seriesID`: BLS ID for series
    - `catalog`: more information about the series (only if you used a valid apikey)
    - `data`: a list of dictionaries with actual data

We'll need a Python function to operate on one of these series results

We'd like it to return a DataFrame with data and Series with metadata

In [20]:
def make_dfs_from_series(series_results):
    """
    Unpack a series response object into data and metadata pandas objects
    
    Parameters
    ----------
    series_results: dict
        A dictionary returned from the `timeseries/data` endpoint of the
        BLS api. An example object for this parameter would be found at
        ``res.json()["Results"]["series"][0]`` where ``res`` is the 
        ``requests.Response`` obtained from interacting with the API endpoint.
    
    Returns
    -------
    data: pd.DataFrame
        A pandas DataFrame containing the actual observations of the data series
    
    metadata: Optional[pd.Series]
        If the ``"catgalog"`` key exists in ``series_results``, then ``metadata``
        is a pandas Series containing the catalog information. If ``"catalog"``
        is found, then this is None    
    """
    series_id = series_results["seriesID"]
    data = pd.DataFrame(series_results["data"]).assign(series_id=series_id)
    if "catalog" in series_results:
        metadata = pd.Series(series_results["catalog"])
    else:
        metadata = None
    return data, metadata

Let's test it out!

In [21]:
data, meta = make_dfs_from_series(js["Results"]["series"][0])
data

,year,period,periodName,latest,value,footnotes,series_id
0,2020,M08,August,true,17301,"[{'code': 'P', 'text': 'Preliminary.'}]",LAUCN040010000000005
1,2020,M07,July,NaN,15976,[{}],LAUCN040010000000005
2,2020,M06,June,NaN,16476,[{}],LAUCN040010000000005
3,2020,M05,May,NaN,17415,[{}],LAUCN040010000000005
4,2020,M04,April,NaN,16923,[{}],LAUCN040010000000005
...,...,...,...,...,...,...,...
123,2010,M05,May,NaN,19802,[{}],LAUCN040010000000005
124,2010,M04,April,NaN,19834,[{}],LAUCN040010000000005
125,2010,M03,March,NaN,19252,[{}],LAUCN040010000000005
126,2010,M02,February,NaN,18996,[{}],LAUCN040010000000005


In [22]:
meta

series_title            Employment: Apache County, AZ (U)
series_id                            LAUCN040010000000005
seasonality                       Not Seasonally Adjusted
survey_name            Local Area Unemployment Statistics
survey_abbreviation                                    LA
measure_data_type                              employment
area                                    Apache County, AZ
area_type                        Counties and equivalents
dtype: object

### Many DataFrames

We're getting closer!

Notice how the `js["Results"]["series"]` was a list?

There will be one of the series results per series requested from the API

We need a function to take a single request `Response` and process the whole list of series data inside

In [24]:
def unpack_response(res: requests.Response):
    """
    Unpack the response for requesting one or more timeseries 
    from the BLS api
    
    Parameters
    ----------
    res: requests.Response
        The object returned from interacting with the ``timeseries/data``
        BLS API endpoint via the reuqests library
    
    Returns
    -------
    datasets: List[Tuple[pd.DataFrame, pd.Series]]
        For each BLS series contained in ``res``, a tuple with the 
        timeseries observations and series metadata will be returned.
        The observations are a pandas DataFrame and the metadata is a
        pandas Series. These pairs of (data, metadata) are returned
        in a list
    
    See Also
    --------
    See ``make_dfs_from_series`` for more information on content
    of output.
    """
    js = res.json()
    return list(map(make_dfs_from_series, js["Results"]["series"]))

Let's test it out

In [25]:
parsed = unpack_response(responses[0])
print(f"Parsed is a {type(parsed)}")
print(f"Parsed has len: {len(parsed)}")

Parsed is a <class 'list'>
Parsed has len: 1


In [26]:
type(parsed[0])

tuple

In [29]:
parsed[0][0]

,year,period,periodName,latest,value,footnotes,series_id
0,2020,M08,August,true,17301,"[{'code': 'P', 'text': 'Preliminary.'}]",LAUCN040010000000005
1,2020,M07,July,NaN,15976,[{}],LAUCN040010000000005
2,2020,M06,June,NaN,16476,[{}],LAUCN040010000000005
3,2020,M05,May,NaN,17415,[{}],LAUCN040010000000005
4,2020,M04,April,NaN,16923,[{}],LAUCN040010000000005
...,...,...,...,...,...,...,...
123,2010,M05,May,NaN,19802,[{}],LAUCN040010000000005
124,2010,M04,April,NaN,19834,[{}],LAUCN040010000000005
125,2010,M03,March,NaN,19252,[{}],LAUCN040010000000005
126,2010,M02,February,NaN,18996,[{}],LAUCN040010000000005


In [28]:
parsed[0][1]

series_title            Employment: Apache County, AZ (U)
series_id                            LAUCN040010000000005
seasonality                       Not Seasonally Adjusted
survey_name            Local Area Unemployment Statistics
survey_abbreviation                                    LA
measure_data_type                              employment
area                                    Apache County, AZ
area_type                        Counties and equivalents
dtype: object

### Many responses

On to the last step...

The `request_for_series` function returns a list of `Response`s

If we pass more than 25 series ids, this list will have more than one item

We need to unpack all the responses, then concatenate the DataFrames and metadata Series

In [30]:
def unpack_all_responses(all_res: List[requests.Response]):
    """
    Given a list of responses from the BLS API, extract and 
    return all data and metadata
    
    Parameters
    ----------
    all_res: List[requests.Response])
        Each item in this list is the result of using ``requests`` to 
        fetch data from ``timeseries/data`` endpoint of the BLS API.
        
    Returns
    -------
    data: pd.DataFrame
        A pandas DataFrame containing all timeseries observations included
        in any of the responses in ``all_res``
    
    metadata: pd.DataFrame
        Detailed metadata about each series, if such metadata exists in the
        response objects
    
    See Also
    --------
    See ``unpack_response`` and ``make_dfs_from_series`` functions
        
    """
    unpacked = []
    for res in all_res:
        unpacked.extend(unpack_response(res))
    
    data_dfs, metadata_series = list(zip(*unpacked))
    data = pd.concat(data_dfs, ignore_index=True)
    metadata = pd.concat([x for x in metadata_series if x is not None], axis=1).T
    return data, metadata

## Unemployment Data

With our functions in place, it is time to get the data for our analysis!

Below we have a list of all series IDs from the BLS report on national unemployment

There are 108 total series

We'll request them all

In [31]:
# all series ids from the BLS unemployment report...
series_ids = [
    "LNU02000000",
    "LNU02000001",
    "LNU02000002",
    "LNS12000000",
    "LNS12000001",
    "LNS12000002",
    "LNU07000000",
    "LNU07000001",
    "LNU07000002",
    "LNS17000000",
    "LNS17000001",
    "LNS17000002",
    "LNU07100000",
    "LNU07100001",
    "LNU07100002",
    "LNS17100000",
    "LNS17100001",
    "LNS17100002",
    "LNU07200000",
    "LNU07200001",
    "LNU07200002",
    "LNS17200000",
    "LNS17200001",
    "LNS17200002",
    "LNU07300000",
    "LNU07300001",
    "LNU07300002",
    "LNS17300000",
    "LNS17300001",
    "LNS17300002",
    "LNU03000000",
    "LNU03000001",
    "LNU03000002",
    "LNS13000000",
    "LNS13000001",
    "LNS13000002",
    "LNU07400000",
    "LNU07400001",
    "LNU07400002",
    "LNS17400000",
    "LNS17400001",
    "LNS17400002",
    "LNU07500000",
    "LNU07500001",
    "LNU07500002",
    "LNS17500000",
    "LNS17500001",
    "LNS17500002",
    "LNU07600000",
    "LNU07600001",
    "LNU07600002",
    "LNS17600000",
    "LNS17600001",
    "LNS17600002",
    "LNU07700000",
    "LNU07700001",
    "LNU07700002",
    "LNS17700000",
    "LNS17700001",
    "LNS17700002",
    "LNU05000000",
    "LNU05000001",
    "LNU05000002",
    "LNS15000000",
    "LNS15000001",
    "LNS15000002",
    "LNU07800000",
    "LNU07800001",
    "LNU07800002",
    "LNS17800000",
    "LNS17800001",
    "LNS17800002",
    "LNU07900000",
    "LNU07900001",
    "LNU07900002",
    "LNS17900000",
    "LNS17900001",
    "LNS17900002",
    "LNU08000000",
    "LNU08000001",
    "LNU08000002",
    "LNS18000000",
    "LNS18000001",
    "LNS18000002",
    "LNU08100000",
    "LNU08100001",
    "LNU08100002",
    "LNS18100000",
    "LNS18100001",
    "LNS18100002",
    "LNU08200000",
    "LNU08200001",
    "LNU08200002",
    "LNS18200000",
    "LNS18200001",
    "LNS18200002",
    "LNU08300000",
    "LNU08300001",
    "LNU08300002",
    "LNS18300000",
    "LNS18300001",
    "LNS18300002",
    "LNU08400000",
    "LNU08400001",
    "LNU08400002",
    "LNS18400000",
    "LNS18400001",
    "LNS18400002",
]

# make requests, get responses
responses = request_for_series(series_ids, "2001", "2020", apikey=apikey)

# validate the responses
[check_response(r) for r in responses]

# extract data and metadata from responses
df, metadata = unpack_all_responses(responses)

Let's see what we have

In [32]:
df

,year,period,periodName,latest,value,footnotes,series_id
0,2020,M09,September,true,147796,[{}],LNU02000000
1,2020,M08,August,NaN,147224,[{}],LNU02000000
2,2020,M07,July,NaN,144492,[{}],LNU02000000
3,2020,M06,June,NaN,142811,[{}],LNU02000000
4,2020,M05,May,NaN,137461,[{}],LNU02000000
...,...,...,...,...,...,...,...
25591,2001,M05,May,NaN,88,[{}],LNS18400002
25592,2001,M04,April,NaN,90,[{}],LNS18400002
25593,2001,M03,March,NaN,88,[{}],LNS18400002
25594,2001,M02,February,NaN,88,[{}],LNS18400002


In [33]:
metadata

,series_title,series_id,seasonality,survey_name,survey_abbreviation,measure_data_type,commerce_industry,occupation,cps_labor_force_status,demographic_age,demographic_ethnic_origin,demographic_race,demographic_gender,demographic_marital_status,demographic_education
0,(Unadj) Employment Level,LNU02000000,Not Seasonally Adjusted,Labor Force Statistics from the Current Popula...,LN,Number in thousands,All Industries,All Occupations,Employed,16 years and over,All Origins,All Races,Both Sexes,All marital statuses,All educational levels
1,(Unadj) Employment Level - Men,LNU02000001,Not Seasonally Adjusted,Labor Force Statistics from the Current Popula...,LN,Number in thousands,All Industries,All Occupations,Employed,16 years and over,All Origins,All Races,Men,All marital statuses,All educational levels
2,(Unadj) Employment Level - Women,LNU02000002,Not Seasonally Adjusted,Labor Force Statistics from the Current Popula...,LN,Number in thousands,All Industries,All Occupations,Employed,16 years and over,All Origins,All Races,Women,All marital statuses,All educational levels
3,(Seas) Employment Level,LNS12000000,Seasonally Adjusted,Labor Force Statistics from the Current Popula...,LN,Number in thousands,All Industries,All Occupations,Employed,16 years and over,All Origins,All Races,Both Sexes,All marital statuses,All educational levels
4,(Seas) Employment Level - Men,LNS12000001,Seasonally Adjusted,Labor Force Statistics from the Current Popula...,LN,Number in thousands,All Industries,All Occupations,Employed,16 years and over,All Origins,All Races,Men,All marital statuses,All educational levels
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,(Unadj) Not in Labor Force to other Marginal O...,LNU08400001,Not Seasonally Adjusted,Labor Force Statistics from the Current Popula...,LN,Number in thousands,All Industries,All Occupations,Not in Labor Force to other Marginal Outflows,16 years and over,All Origins,All Races,Men,All marital statuses,All educational levels
104,(Unadj) Not in Labor Force to other Marginal O...,LNU08400002,Not Seasonally Adjusted,Labor Force Statistics from the Current Popula...,LN,Number in thousands,All Industries,All Occupations,Not in Labor Force to other Marginal Outflows,16 years and over,All Origins,All Races,Women,All marital statuses,All educational levels
105,(Seas) Not in Labor Force to other Marginal Ou...,LNS18400000,Seasonally Adjusted,Labor Force Statistics from the Current Popula...,LN,Number in thousands,All Industries,All Occupations,Not in Labor Force to other Marginal Outflows,16 years and over,All Origins,All Races,Both Sexes,All marital statuses,All educational levels
106,(Seas) Not in Labor Force to other Marginal Ou...,LNS18400001,Seasonally Adjusted,Labor Force Statistics from the Current Popula...,LN,Number in thousands,All Industries,All Occupations,Not in Labor Force to other Marginal Outflows,16 years and over,All Origins,All Races,Men,All marital statuses,All educational levels


And save the data for later usage

In [34]:
metadata.to_parquet("bls_metadata.parquet")

In [35]:
df.to_parquet("bls_data.parquet")